## Random forest implementations

**Random forest** is one of the main supervised learning algorithms that derive from decision trees. The main purpose of gathering individual trees in the way random forests do is to reduce the variance of final estimates at the same time bias is kept under control. Decision trees are highly dependent on the training data, so overfitting is a major concern regarding this learning method. In order to make the final model relies less on the training data, random forests create an ensemble of a large number of *decorrelated trees*, implying that the average of individual predictions can have less variance than a simple bagging, which only attenuates the variability of estimates through the average of identically distributed variables (here, predictions).

Supposing a random forest that should have $B$ trees, each one of them will be grown under the following procedure: a bootstrap sample (indexed by $b$) with size $N$ is drawn from the entire training data (sampling with replacement from all $N$ instances); a tree is then created using this bootstrap sample $b$, where for each split in the tree only a random sample of $m \leq p$ input variables is considered. Given an input vector $x$, the final predictions is the average of predictions of individual trees (majority vote for classification task, average value for regression task and probability estimates).

Fitting a random forest model involves defining (at least) 4 different **hyper-parameters**: minimum node size, $n$, and maximum depth, $J$, (as for a single decision tree), number of trees in the ensemble, $B$, and the number of variables for each split during the growth of each tree $m$. Given the power of this ensemble estimator, it has a relatively low number of hyper-parameters to tune (in comparison with gradient boosting, for instance). Regarding the definition of its hyper-parameters, all discussion related with $n$ and $J$ also applies here. There is some liberty when it comes to define $B$, since random forest can be quite resistant to a large collection of trees. Defining a low value for $m$, however, may be a problem if the learning problem is sparse enough (large collection of inputs $p$ with just a few relevant variables).

The first implementation discussed here follows from this [article](https://towardsdatascience.com/master-machine-learning-random-forest-from-scratch-with-python-3efdd51b6d7a), which first discusses the theoretical definition of the algorithm and then presents an implementation from scratch. The approach of the author considers the creation of three classes: "Node", for declaring a node in a tree; "DecisionTree", for building a single decision tree; and "RandomForest", for fitting and gathering $B$ different decision trees into an ensemble of models.

The **"Node" class** allows the initialization of a node by defining the following attributes: feature and value for the split, data (rows and columns) that goes to the left and right child nodes, the information gain from the split, and the value of response variable. Note that leaf nodes have null values for all attributes, except for the last one; and the opposite applies for decision nodes.

The **"DecisionTree" class** is initialized by declaring the main hyper-parameters of a decision tree: minimum number of samples that follows a split and maximum tree depth. The usage of this class requires initialization and call of *fit* and *predict* methods. But under the hood several methods allows the construction and functioning of a decision tree. First, there is a function for calculating the entropy of a node (*_entropy*), which is used inside another method for calculating the information gain of a split (*_information_gain*), given by the difference between the entropy of the parent node and the weighted average of entropies from the left and right child nodes.

A crucial method of DecisionTree is *_best_split*: it iterates over every available feature and over all possible values of them to find the pair of variable and threshold (or category for categorical inputs) that generates the best split by maximizing the information gain. This method returns a dictionary with the feature for the split, its threshold, the data that goes to the left and the right, and the information gain related with the split.

Then, it comes to the most important method: *_build*, which keeps recursively searching for best splits until at least one of termination criteria is met (all defined by the hyper-parameters of decision trees). This search implies that nodes are constructed using the Node class in the chained manner that usually characterizes the implementation of decision trees. Given that one termination criterium is met, the node will not be further splitted, and only a value of the response variable is defined to it.

Finally, *fit* method takes inputs and output and passes them to the _build method, which assignes the outcome of the method to the root node attribute. The *predict* method also takes inputs to return predicted values of the outcome variable, but uses the *_predict* method to walk the input vector through decision nodes until a leaf is found.

The **RandomForest** class is initialized by declaring all hyper-parameters: minimum node size, maximum depth, and, here, only the number of trees regaring the ensemble construction. This class has three methods. First, *_sample* takes from the training data a bootstrap sample. The *fit* method sequentially creates decision trees by initializing DecisionTree class and by calling its fit method. Finally, the *predict* method iterates over the decision trees in the ensemble and predicts using them by taking the majority vote from all individual predictions.

Some extensions are available to the implementation found here. First, it is possible to use, at each split, a random sample of $m$ variables from all $p$ available. Additionally, random forests can be evaluated or even constructed by using out-of-bag (OOB) error estimates, or any metric calculated under the OOB idea.

**References**
<br>
[Master Machine Learning: Random Forest From Scratch With Python](https://towardsdatascience.com/master-machine-learning-random-forest-from-scratch-with-python-3efdd51b6d7a).
<br>
[The Elements of Statistical Learning](https://web.stanford.edu/~hastie/Papers/ESLII.pdf).

----------------

This notebook first imports all relevant libraries, and then presents an implementation and its demonstration.

**Summary:**
1. [Libraries](#libraries)<a href='#libraries'></a>.
2. [First implementation](#first_implementation)<a href='#first_implementation'></a>.

<a id='libraries'></a>

## Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd "/content/gdrive/MyDrive/Studies/tree_based/Codes"

/content/gdrive/MyDrive/Studies/tree_based/Codes


In [ ]:
import numpy as np
from collections import Counter

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

<a id='first_implementation'></a>

## First implementation

This first implementation follows from this [article](https://towardsdatascience.com/master-machine-learning-random-forest-from-scratch-with-python-3efdd51b6d7a) ([Github](https://github.com/daradecic/BDS-articles/blob/main/014_MML_Random_Forests.ipynb) page of reference).

<a id='first_implementation_codes'></a>

### Codes

#### Class for nodes

In [ ]:
class Node:
    '''
    Helper class which implements a single tree node.
    '''
    def __init__(self, feature=None, threshold=None, data_left=None, data_right=None, gain=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.data_left = data_left
        self.data_right = data_right
        self.gain = gain
        self.value = value

#### Class for the decision tree

In [ ]:
class DecisionTree:
    '''
    Class which implements a decision tree classifier algorithm.
    '''
    def __init__(self, min_samples_split=2, max_depth=5):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None
        
    @staticmethod
    def _entropy(s):
        '''
        Helper function, calculates entropy from an array of integer values.
        
        :param s: list
        :return: float, entropy value
        '''
        # Convert to integers to avoid runtime errors
        counts = np.bincount(np.array(s, dtype=np.int64))
        # Probabilities of each class label
        percentages = counts / len(s)

        # Caclulate entropy
        entropy = 0
        for pct in percentages:
            if pct > 0:
                entropy += pct * np.log2(pct)
        return -entropy
    
    def _information_gain(self, parent, left_child, right_child):
        '''
        Helper function, calculates information gain from a parent and two child nodes.
        
        :param parent: list, the parent node
        :param left_child: list, left child of a parent
        :param right_child: list, right child of a parent
        :return: float, information gain
        '''
        num_left = len(left_child) / len(parent)
        num_right = len(right_child) / len(parent)
        
        # One-liner which implements the previously discussed formula
        return self._entropy(parent) - (num_left * self._entropy(left_child) + num_right * self._entropy(right_child))
    
    def _best_split(self, X, y):
        '''
        Helper function, calculates the best split for given features and target
        
        :param X: np.array, features
        :param y: np.array or list, target
        :return: dict
        '''
        best_split = {}
        best_info_gain = -1
        n_rows, n_cols = X.shape
        
        # For every dataset feature
        for f_idx in range(n_cols):
            X_curr = X[:, f_idx]
            # For every unique value of that feature
            for threshold in np.unique(X_curr):
                # Construct a dataset and split it to the left and right parts
                # Left part includes records lower or equal to the threshold
                # Right part includes records higher than the threshold
                df = np.concatenate((X, y.reshape(1, -1).T), axis=1)
                df_left = np.array([row for row in df if row[f_idx] <= threshold])
                df_right = np.array([row for row in df if row[f_idx] > threshold])

                # Do the calculation only if there's data in both subsets
                if len(df_left) > 0 and len(df_right) > 0:
                    # Obtain the value of the target variable for subsets
                    y = df[:, -1]
                    y_left = df_left[:, -1]
                    y_right = df_right[:, -1]

                    # Caclulate the information gain and save the split parameters
                    # if the current split is better than the previous best
                    gain = self._information_gain(y, y_left, y_right)
                    if gain > best_info_gain:
                        best_split = {
                            'feature_index': f_idx,
                            'threshold': threshold,
                            'df_left': df_left,
                            'df_right': df_right,
                            'gain': gain
                        }
                        best_info_gain = gain
        return best_split
    
    def _build(self, X, y, depth=0):
        '''
        Helper recursive function, used to build a decision tree from the input data.
        
        :param X: np.array, features
        :param y: np.array or list, target
        :param depth: current depth of a tree, used as a stopping criteria
        :return: Node
        '''
        n_rows, n_cols = X.shape
        
        # Check to see if a node should be leaf node
        if n_rows >= self.min_samples_split and depth <= self.max_depth:
            # Get the best split
            best = self._best_split(X, y)
            # If the split isn't pure
            if best['gain'] > 0:
                # Build a tree on the left
                left = self._build(
                    X=best['df_left'][:, :-1], 
                    y=best['df_left'][:, -1], 
                    depth=depth + 1
                )
                right = self._build(
                    X=best['df_right'][:, :-1], 
                    y=best['df_right'][:, -1], 
                    depth=depth + 1
                )
                return Node(
                    feature=best['feature_index'], 
                    threshold=best['threshold'], 
                    data_left=left, 
                    data_right=right, 
                    gain=best['gain']
                )
        # Leaf node - value is the most common target value 
        return Node(
            value=Counter(y).most_common(1)[0][0]
        )
    
    def fit(self, X, y):
        '''
        Function used to train a decision tree classifier model.
        
        :param X: np.array, features
        :param y: np.array or list, target
        :return: None
        '''
        # Call a recursive function to build the tree
        self.root = self._build(X, y)
        
    def _predict(self, x, tree):
        '''
        Helper recursive function, used to predict a single instance (tree traversal).
        
        :param x: single observation
        :param tree: built tree
        :return: float, predicted class
        '''
        # Leaf node
        if tree.value != None:
            return tree.value
        feature_value = x[tree.feature]
        
        # Go to the left
        if feature_value <= tree.threshold:
            return self._predict(x=x, tree=tree.data_left)
        
        # Go to the right
        if feature_value > tree.threshold:
            return self._predict(x=x, tree=tree.data_right)
        
    def predict(self, X):
        '''
        Function used to classify new instances.
        
        :param X: np.array, features
        :return: np.array, predicted classes
        '''
        # Call the _predict() function for every observation
        return [self._predict(x, self.root) for x in X]

#### Class for the random forest

In [ ]:
class RandomForest:
    '''
    A class that implements Random Forest algorithm from scratch.
    '''
    def __init__(self, num_trees=25, min_samples_split=2, max_depth=5):
        self.num_trees = num_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        # Will store individually trained decision trees
        self.decision_trees = []
        
    @staticmethod
    def _sample(X, y):
        '''
        Helper function used for boostrap sampling.
        
        :param X: np.array, features
        :param y: np.array, target
        :return: tuple (sample of features, sample of target)
        '''
        n_rows, n_cols = X.shape
        # Sample with replacement
        samples = np.random.choice(a=n_rows, size=n_rows, replace=True)
        return X[samples], y[samples]
        
    def fit(self, X, y):
        '''
        Trains a Random Forest classifier.
        
        :param X: np.array, features
        :param y: np.array, target
        :return: None
        '''
        # Reset
        if len(self.decision_trees) > 0:
            self.decision_trees = []
            
        # Build each tree of the forest
        num_built = 0
        while num_built < self.num_trees:
            try:
                clf = DecisionTree(
                    min_samples_split=self.min_samples_split,
                    max_depth=self.max_depth
                )
                # Obtain data sample
                _X, _y = self._sample(X, y)
                # Train
                clf.fit(_X, _y)
                # Save the classifier
                self.decision_trees.append(clf)
                num_built += 1
            except Exception as e:
                continue
    
    def predict(self, X):
        '''
        Predicts class labels for new data instances.
        
        :param X: np.array, new instances to predict
        :return: 
        '''
        # Make predictions with every tree in the forest
        y = []
        for tree in self.decision_trees:
            y.append(tree.predict(X))
        
        # Reshape so we can find the most common value
        y = np.swapaxes(a=y, axis1=0, axis2=1)
        
        # Use majority voting for the final prediction
        predictions = []
        for preds in y:
            counter = Counter(preds)
            predictions.append(counter.most_common(1)[0][0])
        return predictions

<a id='first_implementation_codes'></a>

### Demonstration

In [ ]:
# Loading the data and definint input and outcome variables:
iris = load_iris()
X = iris['data']
y = iris['target']

# Train-test split:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating the model object:
model = RandomForest()

# Training the model:
model.fit(X_train, y_train)

# Generating predictions:
preds = model.predict(X_test)

# Accuracy evaluated on test data:
print(f'Accuracy: {accuracy_score(y_test, preds)}.')

Accuracy: 1.0.


#### Comparing performance with Scikit-learn implementation

In [ ]:
# Creating the model object:
sk_model = RandomForestClassifier()

# Training the model:
sk_model.fit(X_train, y_train)

# Generating predictions:
sk_preds = sk_model.predict(X_test)

# Accuracy evaluated on test data:
print(f'Accuracy: {accuracy_score(y_test, sk_preds)}.')

Accuracy: 1.0.
